# Assignment 2: FFNN
**Group 15:** Anil Lingala (akl180001), Kenneth Ly (kll200003), Anshuman Das (axd210123), Parisa Nawar (pxn210032)  
**CS 6320.001**  
**10/17/25**  



# Setup
Importing libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.nn import init
import torch.optim as optim
import math
import random
import os
import time
from tqdm import tqdm
import json
from argparse import ArgumentParser

# FFNN Class
The class representing the FFNN and the forward method we implemented to perform the forwardpass for the network

In [ ]:
unk = '<UNK>'
# Consult the PyTorch documentation for information on the functions used below:
# https://pytorch.org/docs/stable/torch.html
class FFNN(nn.Module):
    def __init__(self, input_dim, h):
        super(FFNN, self).__init__()
        self.h = h
        self.W1 = nn.Linear(input_dim, h)
        self.activation = nn.ReLU() # The rectified linear unit; one valid choice of activation function
        self.output_dim = 5
        self.W2 = nn.Linear(h, self.output_dim)

        self.softmax = nn.LogSoftmax() # The softmax function that converts vectors into probability distributions; computes log probabilities for computational benefits
        self.loss = nn.NLLLoss() # The cross-entropy/negative log likelihood loss taught in class

    def compute_Loss(self, predicted_vector, gold_label):
        return self.loss(predicted_vector, gold_label)

    def forward(self, input_vector):
        # [to fill] obtain first hidden layer representation
        hidden_layer1 = self.W1(input_vector)
        hidden_layer2 = self.activation(hidden_layer1)

        # [to fill] obtain output layer representation
        output_layer = self.W2(hidden_layer2)

        # [to fill] obtain probability dist.
        predicted_vector = self.softmax(output_layer)

        return predicted_vector

# Helper Utility Functions

In [ ]:
# Returns:
# vocab = A set of strings corresponding to the vocabulary
def make_vocab(data):
    vocab = set()
    for document, _ in data:
        for word in document:
            vocab.add(word)
    return vocab

In [ ]:
# Returns:
# vocab = A set of strings corresponding to the vocabulary including <UNK>
# word2index = A dictionary mapping word/token to its index (a number in 0, ..., V - 1)
# index2word = A dictionary inverting the mapping of word2index
def make_indices(vocab):
    vocab_list = sorted(vocab)
    vocab_list.append(unk)
    word2index = {}
    index2word = {}
    for index, word in enumerate(vocab_list):
        word2index[word] = index
        index2word[index] = word
    vocab.add(unk)
    return vocab, word2index, index2word

In [ ]:
# Returns:
# vectorized_data = A list of pairs (vector representation of input, y)
def convert_to_vector_representation(data, word2index):
    vectorized_data = []
    for document, y in data:
        vector = torch.zeros(len(word2index))
        for word in document:
            index = word2index.get(word, word2index[unk])
            vector[index] += 1
        vectorized_data.append((vector, y))
    return vectorized_data

In [ ]:
def load_data(train_data, val_data):
    with open(train_data) as training_f:
        training = json.load(training_f)
    with open(val_data) as valid_f:
        validation = json.load(valid_f)

    tra = []
    val = []
    for elt in training:
        tra.append((elt["text"].split(),int(elt["stars"]-1)))
    for elt in validation:
        val.append((elt["text"].split(),int(elt["stars"]-1)))

    return tra, val

In [ ]:
def log_print(*args, **kwargs):
    print(*args, **kwargs)
    with open("results/test_ffnn.out", "a") as output:
        print(*args, **kwargs, file=output)

# Layers and parameters definitions
We can tweak the arguments and random seeds used by the network

In [ ]:

# Put argumentvalues here
hidden_dim_array = [16, 32, 64]
epochs = 20
trials = 5
do_train = 'yes'

# Path to datasets
train_data_path = "training.json"
val_data_path = "validation.json"
test_data_path = "test.json"

# fix random seeds
random.seed(42)
torch.manual_seed(42)
# os.environ["PYTHONHASHSEED"] = str(42)
# torch.manual_seed(42)
# torch.cuda.manual_seed(42)
# torch.cuda.manual_seed_all(42)
# torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.benchmark = False


# Load and Vectorize Data

In [ ]:
# load data
print("========== Loading data ==========")
train_data, valid_data = load_data(train_data_path, val_data_path) # X_data is a list of pairs (document, y); y in {0,1,2,3,4}
vocab = make_vocab(train_data)
vocab, word2index, index2word = make_indices(vocab)

print("========== Vectorizing data ==========")
train_data = convert_to_vector_representation(train_data, word2index)
valid_data = convert_to_vector_representation(valid_data, word2index)

========== Loading data ==========
========== Vectorizing data ==========


# Training and validation
Perform training on the model given the arguments set above. Then perform validation on the trained model and report the accuracy and validation time for each epoch

In [ ]:
# create results in result directory
os.makedirs("results", exist_ok=True)
val_acc = np.zeros((len(hidden_dim_array), epochs))
train_acc = np.zeros((len(hidden_dim_array), epochs))
train_time = np.zeros((len(hidden_dim_array), epochs))
val_time = np.zeros((len(hidden_dim_array), epochs))

#add results to a file
for hidden_dim_index, hidden_dim in enumerate(hidden_dim_array):
  log_print(f"===FFNN, hidden_dim={hidden_dim}, epochs={epochs}===")
  for i in range(trials):
    log_print(f"Trial {i+1}")
    model = FFNN(input_dim = len(vocab), h = hidden_dim)
    optimizer = optim.SGD(model.parameters(),lr=0.01, momentum=0.9)
    print("========== Training for {} epochs ==========".format(epochs))
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        loss = None
        correct = 0
        total = 0
        start_time = time.time()
        log_print("---Training started for epoch {}---".format(epoch + 1))
        random.shuffle(train_data) # Good practice to shuffle order of training data
        minibatch_size = 16
        N = len(train_data)
        for minibatch_index in tqdm(range(N // minibatch_size)):
            optimizer.zero_grad()
            loss = None
            for example_index in range(minibatch_size):
                input_vector, gold_label = train_data[minibatch_index * minibatch_size + example_index]
                predicted_vector = model(input_vector)
                predicted_label = torch.argmax(predicted_vector)
                correct += int(predicted_label == gold_label)
                total += 1
                example_loss = model.compute_Loss(predicted_vector.view(1,-1), torch.tensor([gold_label]))
                if loss is None:
                    loss = example_loss
                else:
                    loss += example_loss
            loss = loss / minibatch_size
            loss.backward()
            optimizer.step()
        print("Training completed for epoch {}".format(epoch + 1))
        log_print("Training accuracy for epoch {}: {}".format(epoch + 1, correct / total))
        log_print("Training time for this epoch: {}".format(time.time() - start_time))
        train_acc[hidden_dim_index, epoch] += (correct / total)/trials
        train_time[hidden_dim_index, epoch] += (time.time() - start_time)/trials

        loss = None
        correct = 0
        total = 0
        start_time = time.time()
        print("Validation started for epoch {}".format(epoch + 1))
        minibatch_size = 16
        N = len(valid_data)
        for minibatch_index in tqdm(range(N // minibatch_size)):
            optimizer.zero_grad()
            loss = None
            for example_index in range(minibatch_size):
                input_vector, gold_label = valid_data[minibatch_index * minibatch_size + example_index]
                predicted_vector = model(input_vector)
                predicted_label = torch.argmax(predicted_vector)
                correct += int(predicted_label == gold_label)
                total += 1
                example_loss = model.compute_Loss(predicted_vector.view(1,-1), torch.tensor([gold_label]))
                if loss is None:
                    loss = example_loss
                else:
                    loss += example_loss
            loss = loss / minibatch_size
        print("Validation completed for epoch {}".format(epoch + 1))
        log_print("Validation accuracy for epoch {}: {}".format(epoch + 1, correct / total))
        log_print("Validation time for this epoch: {}".format(time.time() - start_time))
        val_acc[hidden_dim_index, epoch] += (correct / total)/trials
        val_time[hidden_dim_index, epoch] += (time.time() - start_time)/trials
    log_print("\n")
log_print("\nFinal Training Accuracy:\n{}".format( train_acc))
log_print("\nFinal Training Time:\n{}".format(train_time))
log_print("\nFinal Validation Accuracy:\n{}".format(val_acc))
log_print("\nFinal Validation Time:\n{}".format(val_time))

===FFNN, hidden_dim=16, epochs=20===
Trial 1
========== Training for 20 epochs ==========
---Training started for epoch 1---


  0%|          | 0/1000 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/nn/modules/module.py:1773: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
100%|██████████| 1000/1000 [00:37<00:00, 26.79it/s]


Training completed for epoch 1
Training accuracy for epoch 1: 0.3460625
Training time for this epoch: 37.341474533081055
Validation started for epoch 1


100%|██████████| 50/50 [00:00<00:00, 114.42it/s]


Validation completed for epoch 1
Validation accuracy for epoch 1: 0.51375
Validation time for this epoch: 0.44609761238098145
---Training started for epoch 2---


...

100%|██████████| 1000/1000 [01:36<00:00, 10.34it/s]


Training completed for epoch 7
Training accuracy for epoch 7: 0.631125
Training time for this epoch: 96.72538781166077
Validation started for epoch 7


100%|██████████| 50/50 [00:00<00:00, 66.93it/s]


Validation completed for epoch 7
Validation accuracy for epoch 7: 0.48875
Validation time for this epoch: 0.754230260848999
---Training started for epoch 8---


 42%|████▏     | 420/1000 [00:41<00:59,  9.77it/s]

# Graphing
Graphs the training and validation accuracy for each parameter for hidden dimension in relation to the epoch number


In [ ]:
# Plot: Training Accuracy vs Epoch for each Hidden Dim
hidden_dim_array = np.array(hidden_dim_array)

epochs = np.arange(1, val_acc.shape[1] + 1)
for i, dim in enumerate(hidden_dim_array):
    plt.plot(epochs, train_acc[i], label=f'Hid Dim {dim}')

plt.xlabel('Epoch')
plt.ylabel('Training Accuracy')
plt.title('Training Accuracy vs Epoch per Hidden Dim')
plt.legend()
plt.grid(True)
plt.gca().xaxis.set_major_locator(plt.MultipleLocator(1)) # Set x-axis major locator to 1
plt.show()

In [ ]:
# Plot: Validation Accuracy vs Epoch for each Hidden Dim

for i, dim in enumerate(hidden_dim_array):
    plt.plot(epochs, val_acc[i], label=f'Hid Dim {dim}')

plt.xlabel('Epoch')
plt.ylabel('Validation Accuracy')
plt.title('Validation Accuracy vs Epoch per Hidden Dim')
plt.legend()
plt.grid(True)
plt.gca().xaxis.set_major_locator(plt.MultipleLocator(1)) # Set x-axis major locator to 1
plt.show()